In [1]:
import os

import tensorflow_text as text
import tensorflow as tf

2023-06-04 12:22:45.571442: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-04 12:22:45.571467: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
BUFFER_SIZE = 20000
SHARD_SIZE = 5000 # number of batches
HYPOTHETICAL_BATCH_SIZE = 32
EXPORT_DIR = 'export/persentence_ds_shard/'

take_size = SHARD_SIZE * HYPOTHETICAL_BATCH_SIZE

In [3]:
try:
    os.makedirs(EXPORT_DIR)
except FileExistsError:
    pass

In [4]:
with open('export/berte_tokenizer.model', 'rb') as f:
    tokenizer = text.SentencepieceTokenizer(model=f.read(), out_type=tf.int32, add_bos=True, add_eos=True)
vocab_size = tokenizer.vocab_size().numpy()

2023-06-04 12:22:47.167185: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-06-04 12:22:47.167212: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-06-04 12:22:47.167235: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mingkaichen-Macmini): /proc/driver/nvidia/version does not exist


In [5]:
all_batches = tf.data.experimental.load('export/persentence_dataset')

In [6]:
def gen():
    for inst in all_batches:
        yield inst, tokenizer.tokenize(inst).shape[0]

training_set = (tf.data.Dataset.from_generator(gen, output_signature=(
                                                    tf.TensorSpec(shape=(), dtype=tf.string),
                                                    tf.TensorSpec(shape=(), dtype=tf.int32),
                                                ))
                                .cache()
                                .shuffle(BUFFER_SIZE))

In [7]:
n = all_batches.cardinality()
full_takes = int(n / take_size)
part_take = n % take_size
print(f'n={n}, full_takes={full_takes}, part_take={part_take}')

n=4306689, full_takes=26, part_take=146689


In [ ]:
for i in range(full_takes):
    model = EXPORT_DIR + f'fshard_{i}'
    tf.data.experimental.save(training_set.take(take_size), model)

tf.data.experimental.save(training_set.take(part_take), EXPORT_DIR + 'pshard')

2023-06-04 12:22:57.294264: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 10109 of 20000
2023-06-04 12:23:07.293871: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 19778 of 20000
2023-06-04 12:23:07.598154: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:415] Shuffle buffer filled.
2023-06-04 12:25:59.499843: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2023-06-04 12:26:09.620581: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 14294 of 20000
2023-06-04 